In [2]:
import pandas as pd
from imblearn.combine import *

In [8]:
%%time
from sklearn.model_selection import GridSearchCV
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from scipy.stats import randint as sp_randint
import numpy as np
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= {'max_depth': [4,5,6],
            'learning_rate': np.arange(0.01, 0.065, 0.005) ,
           'n_estimators' : [210 , 211, 212, 213, 214, 215]}
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']

from imblearn.under_sampling import *
columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X, y = TomekLinks(random_state=0).fit_sample(df_imb_x, df_imb_y)
df_final = pd.DataFrame(X, columns=columns)

X_train, X_test, y_train, y_test = train_test_split(df_final, y, test_size = 0.1, random_state=0, shuffle=True)

xgbc(X_train, X_test, y_train, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBoost :  {'learning_rate': 0.059999999999999991, 'max_depth': 6, 'n_estimators': 215}
train score: -0.5881242838811588
test score: -0.7068810775811291
log loss: 0.7068810775811291

Wall time: 23min 20s
